In [1]:
import pandas as pd
import numpy
import sqlite3
from sklearn.utils import  shuffle
import string

In [2]:
con_1 = sqlite3.connect("/home/musasina/Desktop/projects/ZiraatBankFraudAI/gemini_dataset/collected_data.db")
con_2 = sqlite3.connect("/home/musasina/Desktop/projects/ZiraatBankFraudAI/groq_dataset/collected_data.db")

In [3]:
df_1 = pd.read_sql_query("SELECT * FROM data",con_1)
df_2 = pd.read_sql_query("SELECT * FROM data",con_2)

In [4]:
letters = set(list(string.ascii_letters + "ışİŞüğÜĞçöÇÖ 0123456789"))
def remove_wrong_letters(text:str):
    text_as_list = list(text)
    result_text_list = text_as_list.copy()
    for letter in text_as_list:
        if letter not in letters:
            result_text_list.remove(letter)
    
    return "".join(result_text_list)

In [5]:
df_1["label"] = df_1["label"].convert_dtypes(convert_string=True).apply(lambda x : x.casefold())
df_2["label"] = df_2["label"].convert_dtypes(convert_string=True).apply(lambda x : x.casefold())
df_1["text"] = df_1["text"].convert_dtypes(convert_string=True).apply(lambda x : x.casefold())
df_2["text"] = df_2["text"].convert_dtypes(convert_string=True).apply(lambda x : x.casefold())
df_1["text"] = df_1["text"].convert_dtypes(convert_string=True).apply(remove_wrong_letters)
df_2["text"] = df_2["text"].convert_dtypes(convert_string=True).apply(remove_wrong_letters)

In [6]:
df = pd.concat((df_1,df_2),ignore_index=True)
df_fraud = df[df["label"] == "fraud"]
df_not_fraud = df[df["label"] == "not fraud"]
df = shuffle(pd.concat((df_fraud,df_not_fraud),ignore_index=True),random_state=42)
df = df.dropna()
df.to_csv("/home/musasina/Desktop/projects/ZiraatBankFraudAI/datasets/fraud_text_data.csv",index=False)
